In [1]:
!pip install lxml

You are using pip version 18.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from lxml import etree

In [41]:
#The element class



#Un elemento es el objeto contenedor principal de la API de ElementTree. A la mayor
#parte de la funcionalidad del árbol XML se accede a través de esta clase. Los elementos se crean fácilmente a través de la fábrica de elementos:

root = etree.Element("root")

#Se accede al nombre de la etiqueta XML de los elementos a través de la propiedad de la etiqueta:
print(root.tag)

root


In [42]:
#Los elementos están organizados en una estructura de árbol XML. Para crear elementos secundarios y agregarlos a un elemento primario, puede usar el método append ():
root.append(etree.Element("child1"))

#Sin embargo, esto es tan común que hay una forma más corta y mucho más eficiente de hacer esto: la fábrica de Subelement. 
#Acepta los mismos argumentos que la fábrica de elementos, pero adicionalmente requiere que el padre sea el primer argumento:

child2 = etree.SubElement(root, "child2")
child3 = etree.SubElement(root, "child3")

#Para ver que esto es realmente XML, puede serializar el árbol que ha creado:
print(etree.tostring(root, pretty_print=True))

b'<root>\n  <child1/>\n  <child2/>\n  <child3/>\n</root>\n'


In [43]:
#Elements are lists



#Para que el acceso a estos subelementos sea sencillo y directo, los elementos imitan el comportamiento de las listas normales de Python lo más cerca posible
child = root[0]
print(child.tag)

print(len(root))
root.index(root[1])

for child in root:
    print(child.tag)

child1
3
child1
child2
child3


In [44]:
root.insert(0, etree.Element("child-1"))

In [45]:
start = root[:1]
end = root[-1:]
print(start[0].tag)
print(end[0].tag)

child-1
child3


In [46]:
# Antes de ElementTree 1.3 y lxml 2.0, también puede verificar el valor de verdad de un elemento para ver si tiene hijos, es decir, si la lista de hijos está vacía:
if root:
    print("The root element has children")

The root element has children


C:\Users\DOMINIC\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  


In [47]:
#Esto ya no se admite, ya que las personas tienden a esperar que un "algo" se evalúe como Verdadero y que los Elementos sean "algo", 
#ya sea que tengan hijos o no. Por lo tanto, a muchos usuarios les sorprende que cualquier 
#Elemento se evalúe como Falso en una declaración if como la anterior. En su lugar, use len (elemento), que es más explícito y menos propenso a errores.
print(etree.iselement(child2))
if len(root):
    print("The root element has children")

True
The root element has children


In [48]:
#Hay otro caso importante en el que el comportamiento de los Elementos en lxml (en 2.0 y versiones posteriores) 
#se desvía del de las listas y del de ElementTree original (antes de la versión 1.3 o Python 2.7 / 3.2):
for child in root:
    print(child.tag)
    
root[0] = root[-1]
for child in root:
    print(child.tag)

child-1
child1
child2
child3
child3
child1
child2


In [49]:
#En este ejemplo, el último elemento se mueve a una posición diferente, en lugar de copiarse, es decir, se elimina automáticamente de su posición 
#anterior cuando se coloca en un lugar diferente. En las listas, los objetos pueden aparecer en varias posiciones al mismo tiempo, y la asignación 
#anterior simplemente copiaría la referencia del elemento en la primera posición, de modo que ambos contengan exactamente el mismo elemento:

l = [0, 1, 2, 3]
l[0] = l[-1]
l

[3, 1, 2, 3]

In [50]:
#Tenga en cuenta que en el ElementTree original, un solo objeto Element puede sentarse en cualquier número de lugares en cualquier número de árboles, 
#lo que permite la misma operación de copia que con las listas. El inconveniente obvio es que las modificaciones a dicho Elemento se aplicarán a todos 
#los lugares donde aparece en un árbol, lo cual puede o no estar destinado. La ventaja de esta diferencia es que un Elemento en lxml.etree 
#siempre tiene exactamente un padre, que se puede consultar a través del método getparent (). Esto no se admite en el ElementTree original.

root is root[0].getparent()

True

In [51]:
#Si desea copiar un elemento en una posición diferente en lxml.etree, considere crear una copia profunda 
#independiente utilizando el módulo de copia de la biblioteca estándar de Python:

from copy import deepcopy

element = etree.Element("neu")
element.append(deepcopy(root[1]))

print(element[0].tag)
print([c.tag for c in root])



child1
['child3', 'child1', 'child2']


In [54]:
#Se accede a los hermanos (o vecinos) de un elemento como elementos siguientes y anteriores:
print(etree.tostring(root))
root[0] is root[1].getprevious()
root[1] is root[0].getnext()

b'<root><child3/><child1/><child2/></root>'


True

In [57]:
#Elements contain text



#Los elementos pueden contener texto:

root = etree.Element("root")
root.text = "TExt"

In [58]:
etree.tostring(root)

b'<root>TExt</root>'

In [83]:
#En muchos documentos XML (documentos centrados en datos), este es el único lugar donde se puede encontrar texto. 
#Está encapsulado por una etiqueta de hoja en la parte inferior de la jerarquía del árbol. Sin embargo, si se usa XML para documentos de texto etiquetados 
#como (X) HTML, el texto también puede aparecer entre diferentes elementos, justo en el centro del árbol:Hello
#World Aquí, la etiqueta está rodeada de texto. Esto a menudo se conoce como XML de estilo de documento o contenido mixto. 
#Los elementos soportan esto a través de su propiedad cola. Contiene el texto que sigue directamente al elemento, hasta el siguiente elemento en el árbol XML:

html = etree.Element("html")
body = etree.SubElement(html, "body")
body.text="Cuerpito"

etree.tostring(html)

b'<html><body>Cuerpito</body></html>'

In [84]:
br = etree.SubElement(body, "br")
etree.tostring(html)

b'<html><body>Cuerpito<br/></body></html>'

In [85]:
br.tail = "TAIl"
etree.tostring(html)

b'<html><body>Cuerpito<br/>TAIl</body></html>'

In [86]:
#Las dos propiedades .text y .tail son suficientes para representar cualquier contenido de texto en un documento XML. 
#De esta manera, la API de ElementTree no requiere ningún nodo de texto especial además de la clase Element, que tiende a 
#interferir con bastante frecuencia (como es posible que se sepa de las API de DOM clásicas). Sin embargo, hay casos en que 
#el texto de la cola también se interpone. Por ejemplo, cuando serializa un elemento desde dentro del árbol, no siempre desea 
#que el texto de la cola aparezca en el resultado (aunque aún así querrá el texto de la cola de sus hijos). 
#Para este propósito, la función tostring () acepta el argumento de palabra clave with_tail:

etree.tostring(br)

b'<br/>TAIl'

In [87]:
etree.tostring(br, with_tail=False)

b'<br/>'

In [88]:
pete = etree.SubElement(html, "pete")
pete.text = "petardo"
etree.tostring(html)

b'<html><body>Cuerpito<br/>TAIl</body><pete>petardo</pete></html>'

In [89]:
#If you want to read only the text, i.e. without any intermediate tags, you have to recursively concatenate all text Si desea leer solo el texto, 
#es decir, sin etiquetas intermedias, tiene que concatenar recursivamente todos los atributos de texto y cola en el orden correcto. Nuevamente, 
#la función tostring () viene al rescate, esta vez usando la palabra clave del método:

etree.tostring(html, method="text")

b'CuerpitoTAIlpetardo'

In [90]:
#Using XPath to find text


#Otra forma de extraer el contenido de texto de un árbol es XPath, que también le permite extraer los fragmentos de texto separados en una lista:
print(html.xpath("string()"))

CuerpitoTAIlpetardo


In [91]:
print(html.xpath("//text()"))

['Cuerpito', 'TAIl', 'petardo']


In [92]:
build_text_list = etree.XPath("//text()")
print(build_text_list(html))

['Cuerpito', 'TAIl', 'petardo']


In [93]:
#Tenga en cuenta que un resultado de cadena devuelto por XPath es un objeto especial "inteligente" que conoce sus orígenes. 
#Puede preguntarle de dónde vino a través de su método getparent (), tal como lo haría con Elements:

texts = build_text_list(html)
print(texts[0])

Cuerpito


In [94]:
print(texts[1])

TAIl


In [95]:
print(texts[1].getparent().tag)

br


In [96]:
print(texts[0].is_text)

True


In [97]:
print(texts[1].is_tail)

True


In [98]:
#Si bien esto funciona con los resultados de la función text (), lxml no te dirá el origen de un valor de cadena que fue construido 
#por las funciones XPath string () o concat ():

stringify = etree.XPath("string()")
print(stringify(html))

CuerpitoTAIlpetardo


In [99]:
print(stringify(html).getparent())

None


In [109]:
#Tree iteration


#Para problemas como el anterior, donde desea recorrer recursivamente el árbol y hacer algo con sus elementos, 
#la iteración del árbol es una solución muy conveniente. Los elementos proporcionan un iterador de árbol para este propósito. 
#Produce elementos en el orden de los documentos, es decir, en el orden en que aparecerían sus etiquetas si serializase el árbol a XML:

root = etree.Element("root")
etree.SubElement(root, "child").text = "Child 1"
etree.SubElement(root, "child").text = "Child 2"
etree.SubElement(root, "another").text = "Child 3"

In [110]:
print(etree.tostring(root, pretty_print=True))

b'<root>\n  <child>Child 1</child>\n  <child>Child 2</child>\n  <another>Child 3</another>\n</root>\n'


In [111]:
for element in root.iter():
    print("%s - %s" % (element.tag, element.text))

root - None
child - Child 1
child - Child 2
another - Child 3


In [112]:
#Si sabe que solo le interesa una sola etiqueta, puede pasar su nombre a iter () para que se filtre por usted. 
#A partir de lxml 3.0, también puede pasar más de una etiqueta para interceptar varias etiquetas durante la iteración.

for element in root.iter("child"):
    print("%s - %s" % (element.tag, element.text))

child - Child 1
child - Child 2


In [113]:
for element in root.iter("child", "another"):
    print("%s - %s" % (element.tag, element.text))

child - Child 1
child - Child 2
another - Child 3


In [114]:
#De forma predeterminada, la iteración genera todos los nodos del árbol, incluidas las instancias de ProcessingInstructions, Comments y Entity. 
#Si desea asegurarse de que solo se devuelvan objetos Element, puede pasar la fábrica de Elementos como parámetro de etiqueta:

root.append(etree.Entity("#234"))
root.append(etree.Comment("some comment"))

for element in root.iter():
    if isinstance(element.tag, str):  # or 'str' in Python 3
        print("%s - %s" % (element.tag, element.text))
    else:
        print("SPECIAL: %s - %s" % (element, element.text))

root - None
child - Child 1
child - Child 2
another - Child 3
SPECIAL: &#234; - &#234;
SPECIAL: <!--some comment--> - some comment


In [115]:
for element in root.iter(tag=etree.Element):
    print("%s - %s" % (element.tag, element.text))

root - None
child - Child 1
child - Child 2
another - Child 3


In [116]:
for element in root.iter(tag=etree.Entity):
    print(element.text)

&#234;


In [117]:
#Parsing from strings and files


#lxml.etree supports parsing XML in a number of ways and from all important sources, namely strings, files, URLs (http/ftp) and file-like objects. 
#The main parse functions are fromstring() and parse(), both called with the source as first argument. By default, they use the standard parser, 
#but you can always pass a different parser as second argument.

#the fromstring() function
#The fromstring() function is the easiest way to parse a string:
some_xml_data = "<root>data</root>"
root = etree.fromstring(some_xml_data)

In [118]:
print(root.tag)

root


In [119]:
print(etree.tostring(root))

b'<root>data</root>'


In [121]:
#the XML() function
#La función XML () se comporta como la función fromstring (), pero se usa comúnmente para escribir literales XML directamente en la fuente:
root = etree.XML("<root>data</root>")
print(root.tag)
etree.tostring(root)

root


b'<root>data</root>'

In [122]:
#También hay una función correspondiente HTML () para los literales HTML.
root = etree.HTML("<p>data</p>")
etree.tostring(root)

b'<html><body><p>data</p></body></html>'